In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wHBcPY9dZooyiS_lDa_CIsyBe4i5MWi27dZ7_TqVqbOnhpmGsWP34o
Mounted at /content/gdrive


In [ ]:
import os
import pathlib
import cv2
import glob
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D


%matplotlib inline
np.random.seed(1)

In [ ]:
!unzip -uq "/content/gdrive/My Drive/data_TRAIN.zip"

In [ ]:
data_dir = pathlib.Path("/content/data_TRAIN")

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(750, 400),
  batch_size=32)

Found 598 files belonging to 2 classes.
Using 479 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(750, 400),
  batch_size=32)

Found 598 files belonging to 2 classes.
Using 119 files for validation.


In [ ]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

0.035957 1.0


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 2

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

Epoch 1/3
15/15 [==============================] - 155s 10s/step - loss: 0.1822 - accuracy: 0.9311 - val_loss: 0.1479 - val_accuracy: 0.9580
Epoch 2/3
15/15 [==============================] - 155s 10s/step - loss: 0.1075 - accuracy: 0.9666 - val_loss: 0.1701 - val_accuracy: 0.9664
Epoch 3/3
15/15 [==============================] - 156s 10s/step - loss: 0.0959 - accuracy: 0.9708 - val_loss: 0.1271 - val_accuracy: 0.9664


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_3 (Rescaling)      (None, 750, 400, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 748, 398, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 374, 199, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 372, 197, 32)      9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 186, 98, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 184, 96, 32)       9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 92, 48, 32)       

In [ ]:
from pathlib import Path
X_data = np.ones((6,750,400,3))
files = glob.glob ("/content/Pred/*.jpg")
i=0
for myFile in files:
    print(myFile)
    image = cv2.imread (myFile)
    k=np.array(image)
    p=k.transpose(1, 0, 2).strides
    X_data[i,:]=p
    i+=1

print('X_data shape:', X_data.shape)

X_data shape: (6, 750, 400, 3)


In [ ]:
model.predict_classes(X_data)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([1, 1, 1, 1, 1, 1])

In [ ]:
NAME = "algo"
model.save(f"/content/gdrive/My Drive/{NAME}.model")

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/algo.model/assets


In [ ]:
new_model = tf.keras.models.load_model("/content/gdrive/My Drive/algo.model")